<a href="https://colab.research.google.com/github/Feldt/DeepLearning_NoteBook/blob/main/CIFAR10_Autoencoder_CNN_BaselineReto_SinSolucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np

Cargar el dataset cifar10 en un conjunto inicial de training (imagenes y labels) y test (imágenes y labels)

In [ ]:
(x_train0, y_train0), (x_test0, y_test0) = datasets.cifar10.load_data()

print(f"Forma del conjunto de entrenamiento: {x_train0.shape}")
print(f"Forma del conjunto de prueba: {x_test0.shape}")
print(f"Rango de valores originales: {x_train0.min()} - {x_train0.max()}")


In [ ]:
# lista de etiquetas CIFAR10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### 1. Pre-procesamiento del dataset

a. Convierte los pixeles de las imágenes del conjunto de train y test a tipo float

b. Normaliza, dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.

c. Codifica las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical


In [ ]:
# Convierte los pixeles de las imágenes del conjunto de train y test a tipo float
x_train = x_train0.astype('float32')
x_test = x_test0.astype('float32')

print(f"Despues de conversion a float32:")
print(f"x_train dtype: {x_train.dtype}")
print(f"x_test dtype: {x_test.dtype}")

# dividiendo el valor de todos los pixeles por 255, ya que es el valor máximo de intensidad de cada pixel.
x_train  = x_train / 255
x_test = x_test / 255

print(f"Despues de normalizacion:")
print(f"Rango x_train: {x_train.min()} - {x_train.max()}")
print(f"Rango x_test: {x_test.min()} - {x_test.max()}")


# Codificar las etiquetas y_train y y_test como one-hot utilizando la funcion to_categorical
y_train = keras.utils.to_categorical(y_train0, 10)
y_test = keras.utils.to_categorical(y_test0,10)

print(f"Forma de etiquetas one-hot:")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

Dividimos el test set en 7000 imagenes de validacion (x_val_images) y 3000 de test set (x_test_images)


In [ ]:
# Renombrar por claridad
x_train_images = x_train
x_test_images = x_test


In [ ]:
# Dividir test en validacion y test, 7000 de validacion y 3000 de test
x_val_images = x_test_images[:7000]
x_test_images = x_test_images[7000:]

print(f"Division de conjuntos:")
print(f"Training: {x_train_images.shape}")
print(f"Validation: {x_val_images.shape}")
print(f"Test: {x_test_images.shape}")


Dividimos las etiquetas de validacion (y_val y y_test) de tal forma que correspondan a los conjuntos previos.

In [ ]:
# Dividir etiquetas de validacion
y_val_labels = y_test[:7000]
y_test_labels = y_test[7000:]

print(f"Division de etiquetas:")
print(f"Training labels: {y_train.shape}")
print(f"Validation labels: {y_val_labels.shape}")
print(f"Test labels: {y_test_labels.shape}")


In [ ]:
# Algunos imports complementarios
from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate, Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, MaxPool2D, Flatten, BatchNormalization
import keras.backend as K
from keras.models import Model

In [ ]:

# Modelo encoder-decoder de referencia

print("=== CONSTRUYENDO MODELO ===")


input_img = Input(shape=(32, 32, 3))

# Red encoder
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

print(f"Forma del encoded: {encoded.shape}")


# Red decoder
x = Conv2D(16, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)

In [ ]:
# Modelo Encoder-Decoder
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

### 2. Entrenamiento de autoencoder

a. Entrena el modelo no-supervisado encoder-decoder de tal forma que el modelo aprenda a reconstruir su propio input, con batch_size 64, mínimo 20 épocas.

b. Investiga e implementa el uso de los callbacks de EarlyStopping y ModelCheckpoint

In [ ]:

from keras.callbacks import EarlyStopping, ModelCheckpoint
print("=== ENTRENANDO MODELO BASELINE ===")

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_autoencoder.h5', save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(x_train_images, x_train_images,
                    epochs=20,
                    batch_size=64,
                    shuffle=True,
                    validation_data=(x_val_images, x_val_images),
                    callbacks=[early_stop, checkpoint])

print(f"Training final loss: {history.history['loss'][-1]:.4f}")
print(f"Validation final loss: {history.history['val_loss'][-1]:.4f}")



c. Visualización del proceso de entrenamiento. Grafica Training Loss vs. Validation Loss.


In [ ]:
# Gráfica de Training Loss vs Validation Loss

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoca')
plt.ylabel('Perdida (Loss)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

d. Visualización del input reconstruido

Imprime un conjunto de imágenes originales y comparalas con la imagen reconstruida por el autoencoder.

Utiliza las siguientes funciones de referencia.

In [ ]:
c10test = model.predict(x_test_images)
c10val = model.predict(x_val_images)

In [ ]:
# Funcion para mostrar imagenes originales y reconstruidas

def showOrigDec(orig, dec, num=10):
    import matplotlib.pyplot as plt
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(orig[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # reconstruidas
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(dec[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()


In [ ]:
# Muestra algunas imagenes originales y reconstruidas utilizando tu funcion
showOrigDec(x_test_images, c10test)


A continuación generaremos el modelo encoder del autoencoder. Este modelo lo utilizarás para obtener la representación codificada de los conjuntos originales y con ella entrenar un clasificador.

In [ ]:
# Modelo Encoder
encoder = Model(input_img, encoded)
encoder.summary()

### 3. Extracción de features del autoencoder

Utilizando el **método predict del modelo encoder** extrae las variables que te indicamos a continuación:

a. Codifica el el conjunto de imágenes de entrenamiento utilizando el método predict del encoder, y guárdalo en una variable llamada **gist_train_ae**

b. Codifica el conjunto de imágenes de validación, utilizando el método predict del encoder y guárdalo en una variable llamada **gist_valid_ae**

c. Codifica el conjunto de imágenes de prueba utilizando el método predict del encoder y guárdalo en una variable llamada **gist_test_ae**


In [ ]:
# Completa esta función...
print("=== EXTRAYENDO FEATURES ")
gist_train_ae = encoder.predict(x_train_images)
gist_valid_ae = encoder.predict(x_val_images)
gist_test_ae = encoder.predict(x_test_images)


print(f"Forma de features extraídas (baseline):")
print(f"gist_train_ae: {gist_train_ae.shape}")
print(f"gist_valid_ae: {gist_valid_ae.shape}")
print(f"gist_test_ae: {gist_test_ae.shape}")

print(f"Rango de valores en features:")
print(f"Train features: {gist_train_ae.min():.4f} - {gist_train_ae.max():.4f}")


A continuación definimos un clasificador con una capa convolucional y dos capas densas, que aprenderá a clasificar el input una vez procesado por el codificador.

Puedes utilizar el siguiente clasificador como referencia:

In [ ]:
num_classes = 10
input = Input((gist_train_ae.shape[1], gist_train_ae.shape[2], gist_train_ae.shape[3]))

x = Conv2D(64, 3, padding="same")(input)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(2)(x)
x = Dropout(0.5)(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(num_classes, activation='softmax')(x)

decoder_classifier = Model(input, output)
decoder_classifier.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['acc'])
decoder_classifier.summary()

### 4. Entrenamiento y evaluación del clasificador

a. Entrena el clasificador **decoder_classifier** definido en la fase previa utilizando como input la representación codificada del training set obtenida en el paso previo y como output los labels originales del conjunto de training.

b. Calcula la pérdida de validación del modelo, utilizando la representación codificada de los datos de validación y como ouput los labels originales del conjunto de validación.


In [ ]:

print("=== ENTRENANDO CLASIFICADOR ===")
early_stop_clf = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint_clf = ModelCheckpoint('best_decoder_classifier.h5', save_best_only=True, monitor='val_loss', mode='min')

callbacks = [early_stop_clf, checkpoint_clf]

history_clf = decoder_classifier.fit(gist_train_ae, y_train,
                       validation_data=(gist_valid_ae, y_val_labels),
                       batch_size=64, epochs=20, callbacks=callbacks)

print(f"Classifier - Final training accuracy: {history_clf.history['acc'][-1]:.4f}")
print(f"Classifier - Final validation accuracy: {history_clf.history['val_acc'][-1]:.4f}")


c. Genera predicciones con el modelo clasificador, utiliza el conjunto de test codificado modelo encoder. Guárdalo en la variable pred.




In [ ]:
# Obtener las etiquetas con el clasificador
pred = decoder_classifier.predict(gist_test_ae)

# Convertimos las predicciones a una lista de etiquetas única
pred_classes = np.argmax(pred, axis=1)
print(pred_classes)

print("=== RESULTADOS  ===")
print(f"Forma de predicciones: {pred.shape}")
print(f"Primeras 10 predicciones (clases): {pred_classes[:10]}")
print(f"Primeras 10 etiquetas reales: {np.argmax(y_test_labels[:10], axis=1)}")


d. Evalúa las predicciones del modelo y obten la matriz de confusión.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

cm = confusion_matrix(np.argmax(y_test_labels, axis=1), pred_classes)
accuracy = accuracy_score(np.argmax(y_test_labels, axis=1), pred_classes)

print("Matriz de confusion:\n")
print(cm)
print(f"\nAccuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("Classification Report:\n")
cr=classification_report(y_test_labels.argmax(axis=1), pred.argmax(axis=1), target_names=class_names)
print(cr)


### 5. Mejoras a los modelos

Aplicando los conceptos del curso modifica dichas redes para obtener un mejor accuracy ¿Crees poder lograr un 70% o tal vez 80% de accuracy con tu modelo? OJO: NO está permitido modificar el modelo clasificador.

a.    Experimenta agregando capas, modificando operaciones y modificando las dimensiones de las capas actuales. OJO: Recuerda que para que tu modelo encoder-decoder siga funcionando y puedas reconstruir las imágenes codificadas, las capas de MaxPool del encoder deben de corresponder a las capas UpSample del decoder. Tip: ¿Las capas de pooling ayudan o perjudican a tu modelo?


b.    En una celda de texto, justifica los cambios realizados, a la arquitectura.


c.     Genera la matriz de confusión de tu ensamble de modelos mejorado. Recuerda que debes re-entrenar el clasificador si la arquitectura del autoencoder cambia.



#MEJORAS IMPLEMENTADAS

---

##1. AUTOENCODER:
   - Capas adicionales en cada bloque (64-64, 128-128, 256-256)
   - Skip connections para preservar información
   - Bottleneck más informativo (8x8x128)
   - Misma normalización que baseline  

## 2. CLASIFICADOR:
   - Más filtros iniciales (128 vs 64)
   - Capas adicionales en parte convolucional
   - Red densa más profunda (256->128->10)
   - Dropout moderado (0.25, 0.4)  

## 3. ENTRENAMIENTO:
   - Ligeramente más epochs (35 vs 30)
   - Mismos hiperparámetros base que funcionan
   - Early stopping conservador


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers

In [ ]:
# MODELO - ENFOQUE CONSERVADOR Y EFECTIVO
input_img_improved = Input(shape=(32, 32, 3))

# Encoder - cambios graduales para baseline
x = Conv2D(64, (3, 3), padding='same')(input_img_improved)  # Sin regularizacion excesiva
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), padding='same')(x)  # Capa adicional en primer bloque
x = BatchNormalization()(x)
x = Activation('relu')(x)
skip1 = x  # Skip connection
x = MaxPooling2D((2, 2), padding='same')(x)  # 16x16

x = Conv2D(128, (3, 3), padding='same')(x)  # Incremento gradual de filtros
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
skip2 = x  # Skip connection
x = MaxPooling2D((2, 2), padding='same')(x)  # 8x8

x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Bottleneck más informativo
x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
encoded_improved = Activation('relu')(x)  # 8x8x128

print(f"Forma del encoded: {encoded_improved.shape}")

# Decoder - con skip connections
x = Conv2D(128, (3, 3), padding='same')(encoded_improved)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)  # 16x16
x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Skip connection
x = Add()([x, skip2])
x = Conv2D(128, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = UpSampling2D((2, 2))(x)  # 32x32
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Skip connection
x = Add()([x, skip1])
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded_improved = Activation('sigmoid')(x)  # sigmoid para [0,1]

# Modelo
model_improved = Model(input_img_improved, decoded_improved)
model_improved.compile(optimizer='adam', loss='mse')  # Mantenemos adam simple
model_improved.summary()

print("=== ENTRENANDO MODELO ===")

# Callbacks conservadores
early_stop_improved = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
checkpoint_improved = ModelCheckpoint('best_autoencoder_improved.h5', save_best_only=True, monitor='val_loss', mode='min')

# USAR LOS MISMOS DATOS QUE LA VERSION INICIAL - NO cambia normalizacion
x_train_normalized = x_train_images
x_val_normalized = x_val_images
x_test_normalized = x_test_images

history_improved = model_improved.fit(x_train_normalized, x_train_normalized,
                                    epochs=35,  # Solo un poco más que inicial
                                    batch_size=64,  # Mismo batch size que inicial
                                    shuffle=True,
                                    validation_data=(x_val_normalized, x_val_normalized),
                                    callbacks=[early_stop_improved, checkpoint_improved],
                                    verbose=1)

print(f"Improved Training final loss: {history_improved.history['loss'][-1]:.4f}")
print(f"Improved Validation final loss: {history_improved.history['val_loss'][-1]:.4f}")

# Comparacion de perdidas
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history_clf.history['loss'], label='Base Training', linewidth=2)
plt.plot(history_clf.history['val_loss'], label='Base Validation', linewidth=2)
plt.title('Base Model Loss')
plt.xlabel('Epoca')
plt.ylabel('Perdida')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_improved.history['loss'], label='Improved Training', linewidth=2)
plt.plot(history_improved.history['val_loss'], label='Improved Validation', linewidth=2)
plt.title('Improved Model Loss')
plt.xlabel('Epoca')
plt.ylabel('Perdida')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Predicciones del modelo mejorado
c10test_improved = model_improved.predict(x_test_normalized)
c10val_improved = model_improved.predict(x_val_normalized)

# Comparacion visual de reconstrucciones
print("base: Original vs Reconstruida")
showOrigDec(x_test_images, c10test)
print("Improved: Original vs Reconstruida")
showOrigDec(x_test_images, c10test_improved)

# Encoder mejorado
encoder_improved = Model(input_img_improved, encoded_improved)
encoder_improved.summary()

print("=== EXTRAYENDO FEATURES DEL MODELO MEJORADO ===")

# Extraer features
gist_train_ae_improved = encoder_improved.predict(x_train_normalized)
gist_valid_ae_improved = encoder_improved.predict(x_val_normalized)
gist_test_ae_improved = encoder_improved.predict(x_test_normalized)

print(f"Forma de features extraidas (mejorado):")
print(f"gist_train_ae: {gist_train_ae_improved.shape}")
print(f"gist_valid_ae: {gist_valid_ae_improved.shape}")
print(f"gist_test_ae: {gist_test_ae_improved.shape}")

print(f"Rango de valores en features mejoradas:")
print(f"Train features: {gist_train_ae_improved.min():.4f} - {gist_train_ae_improved.max():.4f}")

# CLASIFICADOR MEJORADO
input_classifier_improved = Input(gist_train_ae_improved.shape[1:])

# Empezar con mas filtros para features mas ricas
x = Conv2D(128, 3, padding="same")(input_classifier_improved)  # Mas filtros que base
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128, 3, padding="same")(x)  # Capa adicional
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(2)(x)
x = Dropout(0.25)(x)  # Menos dropout

x = Conv2D(256, 3, padding="same")(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)  # Mas neuronas que en base
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Dense(128, activation='relu')(x)  # Capa adicional
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

output = Dense(num_classes, activation='softmax')(x)

decoder_classifier_improved = Model(input_classifier_improved, output)
decoder_classifier_improved.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['acc'])

print("=== ENTRENANDO CLASIFICADOR MEJORADO ===")

early_stop_clf_improved = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
checkpoint_clf_improved = ModelCheckpoint('best_decoder_classifier_improved.h5', save_best_only=True, monitor='val_loss', mode='min')

callbacks_improved = [early_stop_clf_improved, checkpoint_clf_improved]

history_clf_improved = decoder_classifier_improved.fit(gist_train_ae_improved, y_train,
                                                     validation_data=(gist_valid_ae_improved, y_val_labels),
                                                     batch_size=64, epochs=30, callbacks=callbacks_improved,
                                                     verbose=1)

print(f"Improved Classifier - Final training accuracy: {history_clf_improved.history['acc'][-1]:.4f}")
print(f"Improved Classifier - Final validation accuracy: {history_clf_improved.history['val_acc'][-1]:.4f}")

# Predicciones del clasificador mejorado
pred_improved = decoder_classifier_improved.predict(gist_test_ae_improved)
pred_classes_improved = np.argmax(pred_improved, axis=1)

print("=== RESULTADOS MODELO MEJORADO ===")
print(f"Forma de predicciones: {pred_improved.shape}")
print(f"Primeras 10 predicciones: {pred_classes_improved[:10]}")

# Evaluacion del modelo mejorado
cm_improved = confusion_matrix(np.argmax(y_test_labels, axis=1), pred_classes_improved)
accuracy_improved = accuracy_score(np.argmax(y_test_labels, axis=1), pred_classes_improved)

print("=== MATRIZ DE CONFUSIÓN MODELO MEJORADO ===")
print(cm_improved)
print(f"\nAccuracy Mejorado: {accuracy_improved:.4f} ({accuracy_improved*100:.2f}%)")
print("\n=== CLASSIFICATION REPORT MODELO MEJORADO ===")
cr_improved = classification_report(y_test_labels.argmax(axis=1), pred_improved.argmax(axis=1), target_names=class_names)
print(cr_improved)

# Comparación final de resultados
print("="*60)
print("COMPARACIoN FINAL DE RESULTADOS")
print("="*60)
print(f"MODELO BASE:")
print(f"  - Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  - Final training loss: {history.history['loss'][-1]:.4f}")
print(f"  - Final validation loss: {history.history['val_loss'][-1]:.4f}")
print(f"  - Classifier val accuracy: {history_clf.history['val_acc'][-1]:.4f}")

print(f"\nIMPROVED MODEL:")
print(f"  - Accuracy: {accuracy_improved:.4f} ({accuracy_improved*100:.2f}%)")
print(f"  - Final training loss: {history_improved.history['loss'][-1]:.4f}")
print(f"  - Final validation loss: {history_improved.history['val_loss'][-1]:.4f}")
print(f"  - Classifier val accuracy: {history_clf_improved.history['val_acc'][-1]:.4f}")

mejora_absoluta = (accuracy_improved - accuracy) * 100
mejora_relativa = ((accuracy_improved - accuracy) / accuracy) * 100

print(f"\nMEJORA ABSOLUTA: {mejora_absoluta:.2f} puntos porcentuales")
print(f"MEJORA RELATIVA: {mejora_relativa:.2f}%")

# Visualización de accuracies del clasificador
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history_clf.history['acc'], label='Training Accuracy', linewidth=2)
plt.plot(history_clf.history['val_acc'], label='Validation Accuracy', linewidth=2)
plt.title(f'Accuracy del Clasificador Base\nFinal Test: {accuracy:.3f}')
plt.xlabel('Epoca')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_clf_improved.history['acc'], label='Training Accuracy', linewidth=2)
plt.plot(history_clf_improved.history['val_acc'], label='Validation Accuracy', linewidth=2)
plt.title(f'Accuracy del Improved Classificador\nFinal Test: {accuracy_improved:.3f}')
plt.xlabel('Epoca')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Matriz de confusion visual
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# matrix de confusion para caso base
im1 = ax1.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax1.set_title(f'Matrix de confusion para caso basen\nAccuracy: {accuracy:.3f}')
ax1.set_xlabel('Predicted Label')
ax1.set_ylabel('True Label')
ax1.set_xticks(range(10))
ax1.set_yticks(range(10))
ax1.set_xticklabels(class_names, rotation=45)
ax1.set_yticklabels(class_names)

for i in range(10):
    for j in range(10):
        text = ax1.text(j, i, cm[i, j], ha="center", va="center", color="white" if cm[i, j] > cm.max()/2 else "black")

# # matrix de confusion improved
im2 = ax2.imshow(cm_improved, interpolation='nearest', cmap=plt.cm.Blues)
ax2.set_title(f'Matrix de confusion Improved \nAccuracy: {accuracy_improved:.3f}')
ax2.set_xlabel('Predicted Label')
ax2.set_ylabel('True Label')
ax2.set_xticks(range(10))
ax2.set_yticks(range(10))
ax2.set_xticklabels(class_names, rotation=45)
ax2.set_yticklabels(class_names)

# improved
for i in range(10):
    for j in range(10):
        text = ax2.text(j, i, cm_improved[i, j], ha="center", va="center", color="white" if cm_improved[i, j] > cm_improved.max()/2 else "black")

plt.tight_layout()
plt.show()


# Analisis de cuales clases mejoraron mas
print("\n" + "="*60)
print("ANALISIS POR CLASE - MEJORAS ESPECIFICAS")
print("="*60)

from sklearn.metrics import precision_recall_fscore_support

# Metricas por clase
precision_base, recall_base, f1_base, support_base = precision_recall_fscore_support(
    np.argmax(y_test_labels, axis=1), pred_classes, average=None)

precision_imp, recall_imp, f1_imp, support_imp = precision_recall_fscore_support(
    np.argmax(y_test_labels, axis=1), pred_classes_improved, average=None)

print(f"{'Clase':<12} {'Base F1':<12} {'Improved F1':<12} {'Mejora':<10}")
print("="*50)
for i, class_name in enumerate(class_names):
    mejora = (f1_imp[i] - f1_base[i]) * 100
    print(f"{class_name:<12} {f1_base[i]:<12.3f} {f1_imp[i]:<12.3f} {mejora:>6.1f}%")

# Guardar modelos
model_improved.save('autoencoder_improved_conservative.h5')
decoder_classifier_improved.save('classifier_improved_conservative.h5')

In [ ]:

# Comparación final de resultados
print("="*60)
print("COMPARACIÓN FINAL DE RESULTADOS")
print("="*60)
print(f"BASELINE MODEL:")
print(f"  - Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"  - Final training loss: {history.history['loss'][-1]:.4f}")
print(f"  - Final validation loss: {history.history['val_loss'][-1]:.4f}")
print(f"  - Classifier val accuracy: {history_clf.history['val_acc'][-1]:.4f}")

print(f"\nIMPROVED MODEL:")
print(f"  - Accuracy: {accuracy_improved:.4f} ({accuracy_improved*100:.2f}%)")
print(f"  - Final training loss: {history_improved.history['loss'][-1]:.4f}")
print(f"  - Final validation loss: {history_improved.history['val_loss'][-1]:.4f}")
print(f"  - Classifier val accuracy: {history_clf_improved.history['val_acc'][-1]:.4f}")

mejora_porcentual = ((accuracy_improved - accuracy) / accuracy) * 100
print(f"\nMEJORA EN ACCURACY: {(accuracy_improved-accuracy)*100:.2f} puntos porcentuales")
print(f"MEJORA PORCENTUAL: {mejora_porcentual:.2f}%")

